<a href="https://colab.research.google.com/github/savatarko/cuda-Smith-Waterman/blob/main/PAProjekat3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import math
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
A = np.array(['C', 'A', 'G', 'C', 'C', 'U', 'C', 'G', 'C', 'U','U','A','G'])
B = np.array(['A', 'A', 'U', 'G', 'C', 'C', 'A', 'U', 'U', 'G','C','C','G', 'G'])
print(A)

['C' 'A' 'G' 'C' 'C' 'U' 'C' 'G' 'C' 'U' 'U' 'A' 'G']


In [ ]:
mod = SourceModule("""
     __global__ void calculate(int *arr, char *A, char *B, int match, int miss, int gap, int n, int m){
       int num = threadIdx.x;
        int i = 1, j=1;
        __syncthreads();
        for(j=1;j<=m;j++){
          if(j-i >= num){
            int myj = j-num;
            int myi = i + num;
            int max, temp;
            max = 0;
            temp = arr[((myi-1)*n + myj) * 2] - gap;
            if(temp > max)
              max = temp;
            temp = arr[((myi)*n + myj - 1) * 2] - gap;
            if(temp > max)
              max = temp;
            if(A[(myi -1) * 4]==B[(myj - 1)*4])
              temp = match;
            else temp = miss * (-1);
            temp = temp + arr[((myi-1)*n + myj - 1) * 2];
            if(temp > max)
              max = temp;
            arr[((myi)*n + myj) * 2] = max;
          }
          __syncthreads();
        }
        j = j - 1;
        for(i;i<=n;i++){
          if(j-i >= num){
            int myj = j-num;
            int myi = i + num;
            int max, temp;
            max = 0;
            temp = arr[((myi-1)*n + myj) * 2] - gap;
            if(temp > max)
              max = temp;
            temp = arr[((myi)*n + myj - 1) * 2] - gap;
            if(temp > max)
              max = temp;
            if(A[(myi -1) * 4]==B[(myj - 1)*4])
              temp = match;
            else temp = miss * (-1);
            temp = temp + arr[((myi-1)*n + myj - 1) * 2];
            if(temp > max)
              max = temp;
            arr[((myi)*n + myj) * 2] = max;
          }
          __syncthreads();
        }
        
        //arr[num * 2] = 1;
     }

      __constant__ char ASH[15] = {'C', 'A', 'G', 'C', 'C', 'U', 'C', 'G', 'C', 'U','U','A','G'};
      __constant__ char BSH[15] = {'A', 'A', 'U', 'G', 'C', 'C', 'A', 'U', 'U', 'G','C','C','G', 'G'};


      __global__ void calculateConstantMem(int *arr, char *A, char *B, int match, int miss, int gap, int n, int m){
       int num = threadIdx.x;
        int i = 1, j=1;
        __syncthreads();
        for(j=1;j<=m;j++){
          if(j-i >= num){
            int myj = j-num;
            int myi = i + num;
            int max, temp;
            max = 0;
            temp = arr[((myi-1)*n + myj) * 2] - gap;
            if(temp > max)
              max = temp;
            temp = arr[((myi)*n + myj - 1) * 2] - gap;
            if(temp > max)
              max = temp;
            if(ASH[(myi -1)]==BSH[(myj - 1)])
              temp = match;
            else temp = miss * (-1);
            temp = temp + arr[((myi-1)*n + myj - 1) * 2];
            if(temp > max)
              max = temp;
            arr[((myi)*n + myj) * 2] = max;
          }
          __syncthreads();
        }
        j = j - 1;
        for(i;i<=n;i++){
          if(j-i >= num){
            int myj = j-num;
            int myi = i + num;
            int max, temp;
            max = 0;
            temp = arr[((myi-1)*n + myj) * 2] - gap;
            if(temp > max)
              max = temp;
            temp = arr[((myi)*n + myj - 1) * 2] - gap;
            if(temp > max)
              max = temp;
            if(ASH[(myi -1)]==BSH[(myj - 1)])
              temp = match;
            else temp = miss * (-1);
            temp = temp + arr[((myi-1)*n + myj - 1) * 2];
            if(temp > max)
              max = temp;
            arr[((myi)*n + myj) * 2] = max;
          }
          __syncthreads();
        }
        
        //arr[num * 2] = 1;
     }





__global__ void calculateSharedMem(int *arr, char *A, char *B, int match, int miss, int gap, int n, int m){

        __shared__ char ASHARED[15];
        __shared__ char BSHARED[15];
       int num = threadIdx.x;
        int i = 1, j=1;
        if(num <14){
            if(num==13){
              BSHARED[num] = BSH[num];
            }
            else{
              ASHARED[num] = ASH[num];
              BSHARED[num] = BSH[num];
            }
        }
        __syncthreads();
        for(j=1;j<=m;j++){
          if(j-i >= num){
            int myj = j-num;
            int myi = i + num;
            int max, temp;
            max = 0;
            temp = arr[((myi-1)*n + myj) * 2] - gap;
            if(temp > max)
              max = temp;
            temp = arr[((myi)*n + myj - 1) * 2] - gap;
            if(temp > max)
              max = temp;
            if(ASH[(myi -1)]==BSH[(myj - 1)])
              temp = match;
            else temp = miss * (-1);
            temp = temp + arr[((myi-1)*n + myj - 1) * 2];
            if(temp > max)
              max = temp;
            arr[((myi)*n + myj) * 2] = max;
          }
          __syncthreads();
        }
        j = j - 1;
        for(i;i<=n;i++){
          if(j-i >= num){
            int myj = j-num;
            int myi = i + num;
            int max, temp;
            max = 0;
            temp = arr[((myi-1)*n + myj) * 2] - gap;
            if(temp > max)
              max = temp;
            temp = arr[((myi)*n + myj - 1) * 2] - gap;
            if(temp > max)
              max = temp;
            if(ASHARED[(myi -1)]==BSHARED[(myj - 1)])
              temp = match;
            else temp = miss * (-1);
            temp = temp + arr[((myi-1)*n + myj - 1) * 2];
            if(temp > max)
              max = temp;
            arr[((myi)*n + myj) * 2] = max;
          }
          __syncthreads();
        }
        
        //arr[num * 2] = 1;
     }






     __global__ void reduce(int *a, int *result){
     
       __shared__ int data[1024];
       int idx = threadIdx.x;
       
       data[idx] = a[idx * 2];
       __syncthreads();
       
       for(unsigned int s=1; s < blockDim.x; s *= 2) { 
       
          if (idx % (2*s) == 0){
            //data[idx] += data[idx + s]; 
            data[idx] = max(data[idx], data[idx + s]);
          }   
          __syncthreads(); 
       }
       
       // write result for this block to global mem
       if (idx == 0) result[0] = data[0];
    }

          __global__ void calculateLarge(int *arr, char *A, char *B, int match, int miss, int gap, int n, int m, int blockn, int blockm, int *currentstate, int blockdelta) {
        //int num = threadIdx.x + blockIdx.x*10;
        int temp;
        if(blockIdx.x == 0){
          currentstate[0] = 1;
          currentstate[1] = 1;
        }
        while(currentstate[0] == 0){
          temp = 1;
        }
        //arr[num] = 1;
        /*
        if(blockIdx.x == 0){
            currentstate = 0;
        }
        */
        int num = threadIdx.x;
        int i = blockIdx.x / blockm, j=blockIdx.x % blockm;
        while(i+j > currentstate[0] - currentstate[1]){
            temp = 1;
        }
        int endi = (i+1) * blockdelta;
        int endj = (j+1) * blockdelta;
        i = i *blockdelta;
        j= j*blockdelta;
        int starti = i;
        int startj = j;
        int numdelta = abs(i - j);
        __syncthreads();
        for(j;j<endj && j <= m;j++){
          if(j-i - numdelta >= num){
            int myj = j-num;
            int myi = i + num;
            if(myi == 0 || myj == 0){
              continue;
            }
            int max, temp;
            max = 0;
            temp = arr[((myi-1)*n + myj) * 2] - gap;
            if(temp > max)
              max = temp;
            temp = arr[((myi)*n + myj - 1) * 2] - gap;
            if(temp > max)
              max = temp;
            if(A[(myi -1) * 4]==B[(myj - 1)*4])
              temp = match;
            else temp = miss * (-1);
            temp = temp + arr[((myi-1)*n + myj - 1) * 2];
            if(temp > max)
              max = temp;
            arr[((myi)*n + myj) * 2] = max;
          }
          __syncthreads();
        }
        j = j - 1;
        for(i;i<endi && i<=n;i++){
          if(j-i- numdelta >= num){
            int myj = j-num;
            int myi = i + num;
            if(myi == 0 || myj == 0){
              continue;
            }
            int max, temp;
            max = 0;
            temp = arr[((myi-1)*n + myj) * 2] - gap;
            if(temp > max)
              max = temp;
            temp = arr[((myi)*n + myj - 1) * 2] - gap;
            if(temp > max)
              max = temp;
            if(A[(myi -1) * 4]==B[(myj - 1)*4])
              temp = match;
            else temp = miss * (-1);
            temp = temp + arr[((myi-1)*n + myj - 1) * 2];
            if(temp > max)
              max = temp;
            arr[((myi)*n + myj) * 2] = max;
          }
          __syncthreads();
        }
        if(starti == 0 || startj == m){
            currentstate[0]++;
        }
    }
""")

<ipython-input-17-cf95948e1673>:1: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(30): warning: expression has no effect

kernel.cu(87): warning: expression has no effect

kernel.cu(156): warning: expression has no effect

kernel.cu(237): warning: expression has no effect

kernel.cu(263): warning: expression has no effect

kernel.cu(210): warning: variable "temp" was set but never used


  mod = SourceModule("""


In [ ]:
def cuda_alloc(*args):
  return [cuda.mem_alloc(arg.nbytes) for arg in args]


def to_cuda(*args):
  cuda_ptrs = cuda_alloc(*args)
  for dst, src in zip(cuda_ptrs, args):
     cuda.memcpy_htod(dst, src)
  return cuda_ptrs

In [ ]:
array = np.zeros([len(A) + 1, len(B) + 1], dtype=int)
mat = np.int32(5)
miss = np.int32(5)
gap = np.int32(5)
#a_gpu = cuda.mem_alloc(A.nbytes)
#b_gpu = cuda.mem_alloc(B.nbytes)
#arr_gpu = cuda.mem_alloc(array.nbytes)
#a_gpu = to_cuda(A)[0]
#b_gpu = to_cuda(B)[0]
#arr_gpu = to_cuda(array)[0]
#cuda.memcpy_htod(a_gpu, A)
#cuda.memcpy_htod(b_gpu, B) 
#cuda.memcpy_htod(arr_gpu, array) 
func = mod.get_function("calculate")
#func(arr_gpu, a_gpu, b_gpu, np.int32(5), np.int32(5), np.int32(5), np.int32(13), np.int32(14), block = (1,1,1), grid = (1,1,1))
func(cuda.InOut(array), cuda.In(A), cuda.In(B), np.int32(3), np.int32(3), np.int32(2), np.int32(15), np.int32(14), block = (15,1,1))

#cuda.memcpy_dtoh(array, arr_gpu)

print(array)

func = mod.get_function("reduce")
max = np.zeros(1, dtype= np.int32)
func(cuda.InOut(array), cuda.InOut(max), block = (14*15, 1, 1), grid = (1,1,1))
print(max[0])

blocknum = math.ceil(len(A)/512) *  math.ceil(len(B)/512)
func = mod.get_function("calculateLarge")

blocknum = np.int32(blocknum)

currentstate = np.int32(0)
currentstate = np.zeros([2], dtype = int)

blocksize = np.int32(512)
blockn = np.int32(math.ceil((len(A) + 1)/blocksize))
blockm = np.int32(math.ceil((len(B) + 1)/blocksize))
#blockn = np.int32(14/n)
#blockm = np.int32(14/m)
n = np.int32(len(A) + 1)
m = np.int32(len(B) + 1)

array = np.zeros([len(A) + 1, len(B) + 1], dtype=int)
func = mod.get_function("calculateConstantMem")
func(cuda.InOut(array), cuda.In(A), cuda.In(B), np.int32(3), np.int32(3), np.int32(2), np.int32(15), np.int32(14), block = (13,1,1))
print(array)

array = np.zeros([len(A) + 1, len(B) + 1], dtype=int)
func = mod.get_function("calculateSharedMem")
func(cuda.InOut(array), cuda.In(A), cuda.In(B), np.int32(3), np.int32(3), np.int32(2), np.int32(15), np.int32(14), block = (13,1,1))
print(array)

print(n)
print(m)
print(currentstate)
blocknum = blockn * blockm
array = np.zeros([len(A) + 1, len(B) + 1], dtype=int)
func = mod.get_function("calculateLarge")
func(cuda.InOut(array), cuda.In(A), cuda.In(B), np.int32(3), np.int32(3), np.int32(2), cuda.In(m), cuda.In(n), cuda.In(blockm), cuda.In(blockn), cuda.InOut(currentstate), np.int32(16),  block = (15,1,1), grid = (1, 1, 1))
#func(cuda.InOut(array), cuda.In(A), cuda.In(B), np.int32(3), np.int32(3), np.int32(2), cuda.In(m), cuda.In(n), cuda.In(blockm), cuda.In(blockn), cuda.InOut(currentstate), cuda.In(blocknum),  block = (20,1,1), grid = (1, 1, 1))
print(array)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  3  3  1  0  0  0  3  3  1  0]
 [ 0  3  3  1  0  1  1  6  4  2  0  1  1  0  0]
 [ 0  1  1  0  4  2  0  4  3  1  5  3  1  4  3]
 [ 0  0  0  0  2  7  5  3  1  0  3  8  6  4  2]
 [ 0  0  0  0  0  5 10  8  6  4  2  6 11  9  7]
 [ 0  0  0  3  1  3  8  7 11  9  7  5  9  8  6]
 [ 0  0  0  1  0  4  6  5  9  8  6 10  8  6  5]
 [ 0  0  0  0  4  2  4  3  7  6 11  9  7 11  9]
 [ 0  0  0  0  2  7  5  3  5  4  9 14 12 10  8]
 [ 0  0  0  3  1  5  4  2  6  8  7 12 11  9  7]
 [ 0  0  0  3  1  3  2  1  5  9  7 10  9  8  6]
 [ 0  3  3  1  0  1  0  5  3  7  6  8  7  6  5]
 [ 0  1  1  0  4  2  0  3  2  5 10  8  6 10  9]]
14
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  3  3  1  0  0  0  3  3  1  0]
 [ 0  3  3  1  0  1  1  6  4  2  0  1  1  0  0]
 [ 0  1  1  0  4  2  0  4  3  1  5  3  1  4  3]
 [ 0  0  0  0  2  7  5  3  1  0  3  8  6  4  2]
 [ 0  0  0  0  0  5 10  8  6  4  2  6 11  9  7]
 [ 0  0  0  3  1  3  8  7 11  9  7  

/usr/local/lib/python3.8/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()
